# Интеллектуальные методы обработки видео

## Задание 1. Scene Change Detector

### Обязательно к прочтению

**Внимание!**

Opencv содержит очень много высокоуровневых функций обработки изображений (например, некоторые алгоритмы компенсации движения, отслеживания объектов, распознавания образов). Использование данной библиотеки в данном задании ограничивается:
* считыванием входного видео
* преобразованием его кадров в другие цветовые пространства
* использованием свёрток Собеля

Использовать библиотеку numpy можно без ограничений.

Если вы хотите использовать функции обработки изображений и видео из другой библиотеки, то оговорите использование этой функции в чате курса.

### Описание входных данных

Выборка для тренировки лежит https://titan.gml-team.ru:5003/sharing/yX8enupJV

Данные о каждом видео лежат в файле *train_dataset\info.json*. Это список из словарей, каждый словарь содержит информацию о расположении видео, о расположении ответов на смены сцен и содержит длину видео

In [21]:
import numpy as np
import cv2 # Для установки opencv воспользуйтесь командой в терминале conda install -c conda-forge opencv
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [22]:
import json
def load_json_from_file(filename):
    with open(filename, "r") as f:
        return json.load(f, strict=False)


def dump_json_to_file(obj, filename, **kwargs):
    with open(filename, "w") as f:
        json.dump(obj, f, **kwargs)

In [23]:
video_dataset = load_json_from_file('train_dataset/info.json')
video_dataset

[{'source': 'video/01.mp4', 'scene_change': 'gt/01.json', 'len': 17531},
 {'source': 'video/03.mp4', 'scene_change': 'gt/03.json', 'len': 3250},
 {'source': 'video/04.mp4', 'scene_change': 'gt/04.json', 'len': 3392},
 {'source': 'video/05.mp4', 'scene_change': 'gt/05.json', 'len': 5662},
 {'source': 'video/07.mp4', 'scene_change': 'gt/07.json', 'len': 3321},
 {'source': 'video/08.mp4', 'scene_change': 'gt/08.json', 'len': 3396},
 {'source': 'video/10.mp4', 'scene_change': 'gt/10.json', 'len': 6096},
 {'source': 'video/14.mp4', 'scene_change': 'gt/14.json', 'len': 2326},
 {'source': 'video/17.mp4', 'scene_change': 'gt/17.json', 'len': 2904},
 {'source': 'video/21.mp4', 'scene_change': 'gt/21.json', 'len': 4898},
 {'source': 'video/22.mp4', 'scene_change': 'gt/22.json', 'len': 7749}]

### Загрузка видео ###

Загрузка видео осуществляется при помощи cv2.VideoCapture. Этот код изменять и дописывать не нужно.

In [24]:
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==False:
            break
        yield frame
    cap.release()

In [25]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))

Что такое frames? Это итератор на кадры видео. Чтобы пройтись по всем кадрам последовательности, воспользуйтесь следующей конструкцией:
*Аккуратно, по одной переменной frames можно пройти только один раз!*

In [26]:
for frame in tqdm(frames):
    pass
for frame in tqdm(frames): # Второй раз уже не будет итерации
    pass

/home/evgenij/miniconda3/envs/evg/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/home/evgenij/miniconda3/envs/evg/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


## Пишем свой простой детектор смен сцен ##

На данном этапе предлагается написать простой Scene Change Detector (SCD) на основе выделения характеристик кадров, подсчёта разницы между кадрами на основе данных характеристик, а также подобрать наиболее оптимальный порог для этих признаков и совместить эти признаки.
Сменой сцен в данной задаче являются только обычные мгновенные смены сцен, без дополнительных эффектов.

В качестве примера приведён простой детектор смен, который считает межкадровую разницу между кадрами.

*Важное замечание. Здесь и далее результатом алгоритма детектора сцен являются **индексы кадров начал сцен**, при этом кадры **нумеруются с 0**. Нулевой кадр в качестве ответа указывать не нужно*

<img src="Hard_cut.jpg">

In [6]:
def baseline_scene_change_detector(frames, threshold=2000, with_vis=False):
    """
    Baseline SCD

    Arguments:
    frames -- iterator on video frames
    threshold -- parameter of your algorithm (optional)
    with_vis -- saving neighboring frames at a scene change (optional)

    Returns:
    scene_changes -- list of scene changes (idx of frames)
    vis -- list of neighboring frames at a scene change (for visualization)
    metric_values -- list of metric values (for visualization)
    """
    
    def pixel_metric(frame, prev_frame):
        # Базовое расстояние между кадрами - среднеквадратическая ошибка между ними
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)

    scene_changes = []
    vis = []
    metric_values = []
    prev_frame = None
    for idx, frame in tqdm(enumerate(frames), leave=False):
        # frame - это кадр
        # idx - это номер кадра
        if prev_frame is not None:
            # Находим расстояние между соседними кадрами
            metric_value = pixel_metric(frame, prev_frame)
            if metric_value > threshold:
                scene_changes.append(idx)
                if with_vis:
                    # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                    if len(vis) < 100:
                        vis.append([prev_frame, frame])
            metric_values.append(metric_value)
        else:
            metric_values.append(0)
        prev_frame = frame
    return scene_changes, vis, metric_values

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
cuts_base = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['cut']
scene_changes_base, vis_base, metric_values_base = baseline_scene_change_detector(frames, with_vis=True)

Посмотрим визуально, насколько сильно алгоритм ошибается, а также на значения метрики

In [6]:
def visualize_metric_error(frame, prev_frame, value):
    fig = plt.figure(figsize=(16,4))
    plt.suptitle('Значение метрики на текущем кадре: {:.4f}'.format(value), fontsize=24)
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(prev_frame[:,:,::-1])
    ax.set_title("Предыдущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(frame[:,:,::-1])
    ax.set_title("Текущий кадр", fontsize=18)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.subplots_adjust(top=0.80)

In [7]:
idx = 1
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# смена сцен

NameError: name 'vis_base' is not defined

In [8]:
idx = 10
visualize_metric_error(vis_base[idx][0], vis_base[idx][1], metric_values_base[scene_changes_base[idx]])
# ошибается, это не смена сцен

NameError: name 'vis_base' is not defined

In [9]:
def visualize_metric_values(metric_values, threshold, cuts = None):
    sns.set()
    plt.figure(figsize=(16, 8))
    plt.plot(metric_values, label='Значение метрики на кадрах')
    plt.xlabel('Номер кадра')
    plt.ylabel('Значение метрики')
    plt.hlines(y=threshold, xmin=0, xmax=len(metric_values), linewidth=2, color='r', label='Пороговое значение')
    
    if cuts is not None:
        for cut in cuts:
            plt.axvline(x=cut, color='k', linestyle=':', linewidth=0.5, label='Смена сцены')

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [10]:
visualize_metric_values(metric_values_base, 2000, cuts_base)

NameError: name 'metric_values_base' is not defined

**Как видим, очень плохо подобран порог, да и сам признак, похоже, сильно зашумлён. Попробуйте что-то своё!**

## Ваше решение ##

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector(frames)`  
* Строку (# GRADED FUNCTION: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [27]:
# GRADED FUNCTION: scene_change_detector

import joblib
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import AdaBoostClassifier
def scene_change_detector(frames, with_vis=False, cuts = None):
    #print(threshold, Sobel_t, hist_threshold)
    
    scene_changes = []
    vis = []
    metric_values = []
    
    ### START CODE HERE ###
    # Ваши внешние переменные
    prev_frame = None
    metric_values1 = []
    model = joblib.load('model_new.pkl')
    def pixel_metric(frame, prev_frame):
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)
    def brightness_metric(frame, prev_frame):
        return np.abs(np.mean(frame) - np.mean(prev_frame))
    def brightness_differense_metric(frame, prev_frame):
        return np.abs(np.mean(frame - prev_frame))
    def std_metric(frame, prev_frame):
        return np.abs(np.std(frame) - np.std(prev_frame))
    def std_diff_metric(frame, prev_frame):
        return np.abs(np.std(frame - prev_frame))
    def hist_metric(frame, prev_frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        hist1 = cv2.calcHist([frame], [0], None, [64], [1, 64])
        hist2 = cv2.calcHist([prev_frame], [0], None, [64], [1, 64])
        return np.abs((hist1 - hist2)).ravel()
    def hist_corel_metric(frame, prev_frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        hist1 = cv2.calcHist([frame], [0], None, [64], [1, 64])
        hist2 = cv2.calcHist([prev_frame], [0], None, [64], [1, 64])
        return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    def hist_feature_metric(frame, prev_frame, gray=False, crop_x = 10, crop_y = 10):
        height, width = 1, 1
        if gray == False:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
            height, width = frame.shape
        else:
            height, width = frame.shape
        cut = np.zeros((crop_x * crop_y, 4))
        ind = 0
        for ih in range(crop_y):
            for iw in range(crop_x):
                x = width//crop_x * iw 
                y = height//crop_y * ih
                h = height // crop_y
                w = width // crop_x
                tmp_frame = frame[y:y+h, x:x+w]
                tmp_prev_frame = prev_frame[y:y+h, x:x+w]
                a = cv2.calcHist([tmp_frame], [0], None, [64], [1, 64])
                b = cv2.calcHist([tmp_prev_frame], [0], None, [64], [1, 64])
                cut[ind][0] = cv2.compareHist(a, b, cv2.HISTCMP_CORREL)
                cut[ind][1] = cv2.compareHist(a, b, cv2.HISTCMP_INTERSECT)
                cut[ind][2] = cv2.compareHist(a, b, cv2.HISTCMP_KL_DIV)
                tmp_frame = tmp_frame / 255
                tmp_prev_frame = tmp_prev_frame / 255
                tmp = brightness_metric(tmp_frame, tmp_prev_frame) / 4
                std1 = np.var(tmp_frame) / 2
                std2 = np.var(tmp_prev_frame)
                if std1 == 0 or std2 == 0:
                    cut[ind][3] = (tmp + std1 * std2) ** 2 / 0.112
                else:
                    cut[ind][3] = (tmp + std1 * std2) ** 2 / (std1 * std2 * 2)
                ind += 1
        return list(np.mean(cut, axis = 0))
    def cos_metric(frame, prev_frame):
        tmp1 = np.sum(frame * prev_frame)
        tmp = np.sqrt(np.sum(prev_frame**2))
        tmp = np.sqrt(tmp * np.sum(frame**2))
        if tmp == 0:
            tmp = tmp1 / 0.0123
        else:
            tmp = tmp1 / tmp
        return tmp
    def hist_eq(frame):
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(3,3))
        cl = clahe.apply(l)
        limg = cv2.merge((cl,a,b))
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        return final
    def laplasian(frame):
        frame1 = cv2.GaussianBlur(frame, (3, 3), 0)
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame1 = cv2.Laplacian(frame1, cv2.CV_16S, ksize=5)
        abs_dst1 = cv2.convertScaleAbs(frame1)
        return abs_dst1
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        
        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        if prev_frame is not None:
            fr = frame
            fr_prev = prev_frame
            metrics = []
            #metrics.append(pixel_metric(fr, fr_prev)) 
            #metrics.append(brightness_metric(fr, fr_prev))
            #metrics.append(brightness_differense_metric(fr, fr_prev))
            metrics.append(std_metric(fr, fr_prev))
            metrics.append(std_diff_metric(fr, fr_prev))
            metrics.append(hist_corel_metric(fr, fr_prev))
            tmp = hist_feature_metric(fr, fr_prev)
            metrics.append(tmp[1])
            metrics.append(tmp[3])
            arr = hist_metric(fr, fr_prev)
            metrics.append(np.mean(arr))
            #metrics.append(np.std(arr))
            metrics.append(cos_metric(fr, fr_prev))
            
            fr = laplasian(frame)
            fr_prev = laplasian(prev_frame)
            metrics.append(pixel_metric(fr, fr_prev)) 
            ######metrics.append(brightness_metric(fr, fr_prev))
            #metrics.append(brightness_differense_metric(fr, fr_prev))
            #metrics.append(std_metric(fr, fr_prev))
            metrics.append(std_diff_metric(fr, fr_prev))
            #metrics.append(cos_metric(fr, fr_prev))
            tmp = hist_feature_metric(fr, fr_prev, gray=True)
            metrics.append(tmp[0])
            metrics.append(tmp[2])
            metrics.append(tmp[3])
            #metrics.append(hist_corel_metric(fr, fr_prev))
            #metrics += list(hist_metric(fr, fr_prev))
            
            fr = hist_eq(frame)
            fr_prev = hist_eq(prev_frame)
            metrics.append(pixel_metric(fr, fr_prev)) 
            #metrics.append(brightness_metric(fr, fr_prev))
            metrics.append(brightness_differense_metric(fr, fr_prev))
            metrics.append(std_metric(fr, fr_prev))
            #metrics.append(std_diff_metric(fr, fr_prev))
            #metrics.append(hist_corel_metric(fr, fr_prev))
            tmp = hist_feature_metric(fr, fr_prev)
            metrics.append(tmp[1])
            metrics.append(tmp[3])
            arr = hist_metric(fr, fr_prev)
            metrics.append(np.mean(arr))
            #metrics.append(np.std(arr))
            metrics.append(cos_metric(fr, fr_prev))

            
            fr = laplasian(fr)
            fr_prev = laplasian(fr_prev)
            metrics.append(pixel_metric(fr, fr_prev)) 
            metrics.append(brightness_metric(fr, fr_prev))
            #metrics.append(brightness_differense_metric(fr, fr_prev))
            #metrics.append(std_metric(fr, fr_prev))
            metrics.append(std_diff_metric(fr, fr_prev))
            #metrics.append(cos_metric(fr, fr_prev))
            #metrics += hist_feature_metric(fr, fr_prev, gray=True)
            #metrics.append(hist_corel_metric(fr, fr_prev))
            #metrics += list(hist_metric(fr, fr_prev))
            
            
            #
            metrics = np.array(metrics).reshape(1, -1)
            lol = SimpleImputer(missing_values=np.inf, strategy='mean')
            lol = lol.fit(metrics)
            metrics = lol.transform(metrics)
            lol = SimpleImputer(missing_values=np.nan, strategy='mean')
            lol = lol.fit(metrics)
            metrics = lol.transform(metrics)
            lol = SimpleImputer(missing_values=-np.inf, strategy='mean')
            lol = lol.fit(metrics)
            metrics = lol.transform(metrics)
            metrics = np.array(metrics, dtype='float32')
            #print(metrics)
            
            ans = model.predict(metrics)
            '''if idx in cuts:
                metrics.append(1)
            else:
                metrics.append(0)'''
            #dataset.loc[idx] = metrics
            if ans == 1:
                scene_changes.append(idx)
                if with_vis:
                    # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                    if len(vis) < 100:
                        vis.append([prev_frame, frame])
        prev_frame = frame
        ###  END CODE HERE  ###
    return scene_changes, vis, metric_values#, metric_values1, metric_values2

In [15]:
import pandas as pd

In [20]:
for name in ['03', '04', '05', '07', '08', '10', '14', '17', '21', '22']:
    fd = pd.read_csv(f'tmp_new/video_{name}_dataset.csv')
    print(np.isinf(fd.values).mean(), np.isnan(fd.values).mean())

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0


In [19]:
fd = pd.read_csv('tmp_new/video_03_dataset.csv')
np.isinf(fd.values).mean()

0.0

In [ ]:
frames = read_video(os.path.join('train_dataset', 'video', '03.mp4'))
cuts = load_json_from_file(os.path.join('train_dataset', 'gt', '03.json'))['cut']
scene_changes, vis, metric_values = scene_change_detector(frames, with_vis=True)

#### Обратите внимание на скорость работы алгоритма! ####
Если вычислять признаки без циклов по пикселям, а пользоваться методами из numpy, то скорость будет не медленнее 7-8 кадров в секунду.
Например, вы можете использовать функцию `np.histogram` или `cv2.calcHist` для подсчёта гистограмм, а `cv2.Sobel` для применения оператора Собеля к кадру.

In [ ]:
#Посмотрим на найденные смены сцен
idx = 1 
visualize_metric_error(vis[idx][0], vis[idx][1], metric_values[scene_changes[idx]])

In [ ]:
#Посмотрим на значения метрики
visualize_metric_values(metric_values, 2000, cuts)

## Подсчёт метрики F1-Score##

Чтобы оценивать алгоритм и научиться сравнивать несколько алгоритмов, нужна метрика качества. В данной задаче для оценки качества алгоритма используется F1-Score. Преимущества использования этой метрики к текущей постановке задачи смены сцен были рассказаны на лекции, напишем только формулы:
$$precision = \frac{tp}{tp+fp}$$
$$recall = \frac{tp}{tp+fn}$$
$$F = 2 * \frac{precision * recall}{precision+recall}$$

На всякий случай опишем как именно происходит подсчёт метрики для видео

1) Сначала из выборки удаляются все кадры, которые по разметке либо являются сложными переходами между сценами, либо помечены как сложные для анализа и разметки (например, титры/обилие компьютерной графики и т.п)


2) Затем для оставшихся кадров уже подсчитывается F1_Score

In [28]:
#Эти пять клеток кода править не нужно
def calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    for scd in predicted_scd:
        if scd in true_scd:
            tp += 1
        elif scd not in not_to_use_frames:
            fp += 1
    for scd in true_scd:
        if scd not in predicted_scd:
            fn += 1
    tn = scene_len - len(not_to_use_frames) - tp - fp - fn
    return tp, fp, tn, fn

In [29]:
def calculate_precision(tp, fp, tn, fn):
    return tp / max(1, (tp + fp))

In [30]:
def calculate_recall(tp, fp, tn, fn):
    return tp / max(1, (tp + fn))

In [31]:
def f1_score(true_scd, predicted_scd, scene_len, not_to_use_frames=set()):
    tp, fp, tn, fn = calculate_matrix(true_scd, predicted_scd, scene_len, not_to_use_frames)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [32]:
def f1_score_matrix(tp, fp, tn, fn):
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

## Тестируем разработанный метод сразу на нескольких видео ##

Проверим, насколько хорошо работает разработанный метод. *Учтите, что итоговое тестирование будет производиться на аналогичном, но недоступном вам наборе видео, но все параметры алгоритмов должны быть указаны вами (иными словами - подобраны на тренировочном наборе).*

In [33]:
def run_scene_change_detector_all_video(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        # Загружаем видео, его длину и смены сцен
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))
        
        # Составляем список сцен, которые не будут тестироваться
        not_use_frames = set()
        for type_scene_change in ['trash', 'fade', 'dissolve']:
            for bad_scene_range in true_scene_changes.get(type_scene_change, []):
                not_use_frames.update(list(range(bad_scene_range[0], bad_scene_range[1] + 1)))
        
        predicted_scene_changes, _, _ = scene_change_detector(frames)
        
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix(
            true_scene_changes['cut'],
            predicted_scene_changes,
            video_len,
            not_use_frames
        )
        
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn 
        print(param_log['f1_score_{}'.format(video_info['source'])])
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [34]:
video_dataset = 'train_dataset'

Данная функция поможет вам посмотреть, на каких видео и на сколько ошибается ваш метод. Прогнать метод на отдельном видео и детально посмотреть кадры вы могли выше.

Кроме того, с помощью этой функции вы можете подобрать оптимальные параметры для метода.

In [ ]:
#Протестируем базовый метод
run_scene_change_detector_all_video(baseline_scene_change_detector, video_dataset)

In [35]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video(scene_change_detector, video_dataset)

/home/evgenij/miniconda3/envs/evg/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/evgenij/miniconda3/envs/evg/lib/python3.7/site-packages/ipykernel_launcher.py:99: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

Когда вы смотрите на результат, обращайте внимание на **_mean_f1_score**  
Именно по этой метрике будет производится финальное оценивание.

## Бонусное задание: распознавание смен сцен типа "наложения"

На практике кроме катов часто встречаются смены сцен, где происходит "наложение" одной сцены на другую:

<img src="Dissolve.jpg">

## Ваше решение ##

* В качестве решения вы должны прикрепить функцию ниже. Все пороги должны быть указаны внутри функции.  
Т.е. должен быть возможен вызов:  
`scene_changes, vis, metric_values = scene_change_detector_dissolve(frames)`  
* Строку (# GRADED FUNCTION: [function name]) менять **нельзя**. Она будет использоваться при проверке вашего решения.
* Ячейка должна содержать только **одну** функцию.

In [ ]:
# GRADED FUNCTION: scene_change_detector_dissolve

import joblib
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import AdaBoostClassifier
def scene_change_detector_dissolve(frames, with_vis=False, cuts = None):
    #print(threshold, Sobel_t, hist_threshold)
    
    scene_changes = []
    vis = []
    metric_values = []
    
    ### START CODE HERE ###
    # Ваши внешние переменные
    prev_frame = None
    metric_values1 = []
    model = joblib.load('model_new.pkl')
    def pixel_metric(frame, prev_frame):
        return np.mean((frame.astype(np.int32) - prev_frame) ** 2)
    def brightness_metric(frame, prev_frame):
        return np.abs(np.mean(frame) - np.mean(prev_frame))
    def brightness_differense_metric(frame, prev_frame):
        return np.abs(np.mean(frame - prev_frame))
    def std_metric(frame, prev_frame):
        return np.abs(np.std(frame) - np.std(prev_frame))
    def std_diff_metric(frame, prev_frame):
        return np.abs(np.std(frame - prev_frame))
    def hist_metric(frame, prev_frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        hist1 = cv2.calcHist([frame], [0], None, [64], [1, 64])
        hist2 = cv2.calcHist([prev_frame], [0], None, [64], [1, 64])
        return np.abs((hist1 - hist2)).ravel()
    def hist_corel_metric(frame, prev_frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        hist1 = cv2.calcHist([frame], [0], None, [64], [1, 64])
        hist2 = cv2.calcHist([prev_frame], [0], None, [64], [1, 64])
        return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    def hist_feature_metric(frame, prev_frame, gray=False, crop_x = 10, crop_y = 10):
        height, width = 1, 1
        if gray == False:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
            height, width = frame.shape
        else:
            height, width = frame.shape
        cut = np.zeros((crop_x * crop_y, 4))
        ind = 0
        for ih in range(crop_y):
            for iw in range(crop_x):
                x = width//crop_x * iw 
                y = height//crop_y * ih
                h = height // crop_y
                w = width // crop_x
                tmp_frame = frame[y:y+h, x:x+w]
                tmp_prev_frame = prev_frame[y:y+h, x:x+w]
                a = cv2.calcHist([tmp_frame], [0], None, [64], [1, 64])
                b = cv2.calcHist([tmp_prev_frame], [0], None, [64], [1, 64])
                cut[ind][0] = cv2.compareHist(a, b, cv2.HISTCMP_CORREL)
                cut[ind][1] = cv2.compareHist(a, b, cv2.HISTCMP_INTERSECT)
                cut[ind][2] = cv2.compareHist(a, b, cv2.HISTCMP_KL_DIV)
                tmp_frame = tmp_frame / 255
                tmp_prev_frame = tmp_prev_frame / 255
                tmp = brightness_metric(tmp_frame, tmp_prev_frame) / 4
                std1 = np.var(tmp_frame) / 2
                std2 = np.var(tmp_prev_frame)
                if std1 == 0 or std2 == 0:
                    cut[ind][3] = (tmp + std1 * std2) ** 2 / 0.112
                else:
                    cut[ind][3] = (tmp + std1 * std2) ** 2 / (std1 * std2 * 2)
                ind += 1
        return list(np.mean(cut, axis = 0))
    def cos_metric(frame, prev_frame):
        tmp1 = np.sum(frame * prev_frame)
        tmp = np.sqrt(np.sum(prev_frame**2))
        tmp = np.sqrt(tmp * np.sum(frame**2))
        if tmp == 0:
            tmp = tmp1 / 0.0123
        else:
            tmp = tmp1 / tmp
        return tmp
    def hist_eq(frame):
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(3,3))
        cl = clahe.apply(l)
        limg = cv2.merge((cl,a,b))
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        return final
    def laplasian(frame):
        frame1 = cv2.GaussianBlur(frame, (3, 3), 0)
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame1 = cv2.Laplacian(frame1, cv2.CV_16S, ksize=5)
        abs_dst1 = cv2.convertScaleAbs(frame1)
        return abs_dst1
    ###  END CODE HERE  ###
    
    for idx, frame in tqdm(enumerate(frames), leave=False):
        
        ### START CODE HERE ###
        # Основная часть вашего алгоритма
        if prev_frame is not None:
            fr = frame
            fr_prev = prev_frame
            metrics = []
            #metrics.append(pixel_metric(fr, fr_prev)) 
            #metrics.append(brightness_metric(fr, fr_prev))
            #metrics.append(brightness_differense_metric(fr, fr_prev))
            metrics.append(std_metric(fr, fr_prev))
            metrics.append(std_diff_metric(fr, fr_prev))
            metrics.append(hist_corel_metric(fr, fr_prev))
            tmp = hist_feature_metric(fr, fr_prev)
            metrics.append(tmp[1])
            metrics.append(tmp[3])
            arr = hist_metric(fr, fr_prev)
            metrics.append(np.mean(arr))
            #metrics.append(np.std(arr))
            metrics.append(cos_metric(fr, fr_prev))
            
            fr = laplasian(frame)
            fr_prev = laplasian(prev_frame)
            metrics.append(pixel_metric(fr, fr_prev)) 
            ######metrics.append(brightness_metric(fr, fr_prev))
            #metrics.append(brightness_differense_metric(fr, fr_prev))
            #metrics.append(std_metric(fr, fr_prev))
            metrics.append(std_diff_metric(fr, fr_prev))
            #metrics.append(cos_metric(fr, fr_prev))
            tmp = hist_feature_metric(fr, fr_prev, gray=True)
            metrics.append(tmp[0])
            metrics.append(tmp[2])
            metrics.append(tmp[3])
            #metrics.append(hist_corel_metric(fr, fr_prev))
            #metrics += list(hist_metric(fr, fr_prev))
            
            fr = hist_eq(frame)
            fr_prev = hist_eq(prev_frame)
            metrics.append(pixel_metric(fr, fr_prev)) 
            #metrics.append(brightness_metric(fr, fr_prev))
            metrics.append(brightness_differense_metric(fr, fr_prev))
            metrics.append(std_metric(fr, fr_prev))
            #metrics.append(std_diff_metric(fr, fr_prev))
            #metrics.append(hist_corel_metric(fr, fr_prev))
            tmp = hist_feature_metric(fr, fr_prev)
            metrics.append(tmp[1])
            metrics.append(tmp[3])
            arr = hist_metric(fr, fr_prev)
            metrics.append(np.mean(arr))
            #metrics.append(np.std(arr))
            metrics.append(cos_metric(fr, fr_prev))

            
            fr = laplasian(fr)
            fr_prev = laplasian(fr_prev)
            metrics.append(pixel_metric(fr, fr_prev)) 
            metrics.append(brightness_metric(fr, fr_prev))
            #metrics.append(brightness_differense_metric(fr, fr_prev))
            #metrics.append(std_metric(fr, fr_prev))
            metrics.append(std_diff_metric(fr, fr_prev))
            #metrics.append(cos_metric(fr, fr_prev))
            #metrics += hist_feature_metric(fr, fr_prev, gray=True)
            #metrics.append(hist_corel_metric(fr, fr_prev))
            #metrics += list(hist_metric(fr, fr_prev))
            
            
            #
            metrics = np.array(metrics).reshape(1, -1)
            lol = SimpleImputer(missing_values=np.inf, strategy='mean')
            lol = lol.fit(metrics)
            metrics = lol.transform(metrics)
            lol = SimpleImputer(missing_values=np.nan, strategy='mean')
            lol = lol.fit(metrics)
            metrics = lol.transform(metrics)
            lol = SimpleImputer(missing_values=-np.inf, strategy='mean')
            lol = lol.fit(metrics)
            metrics = lol.transform(metrics)
            #print(metrics)
            
            ans = model.predict(metrics)
            '''if idx in cuts:
                metrics.append(1)
            else:
                metrics.append(0)'''
            #dataset.loc[idx] = metrics
            if ans == 1:
                scene_changes.append(idx)
                if with_vis:
                    # Кадры в памяти занимают много места, поэтому сохраним лишь первые 100 срабатываний
                    if len(vis) < 100:
                        vis.append([prev_frame, frame])
        prev_frame = frame
        ###  END CODE HERE  ###
    return scene_changes, vis, metric_values#, metric_values1, metric_values2

В качестве метрики качества используется видоизменённый f1-score:

Так как смена сцен не происходит за один кадр, попаданием считается попадание ответа смены сцен в отрезок, где происходит наложение.  
**Обратите внимание**, что несколько раз указывать одну смену сцен не нужно.

Попадание вне отрезков смен сцен путём наложения считается как false positive, не попадание в указанный отрезок - как false negative

In [ ]:
#Эти три клетки кода править не нужно
def calculate_matrix_dissolve(true_scd, predicted_scd, scene_len):
    tp, fp, tn, fn = 0, 0, 0, 0
    scene_len = scene_len
    checked_dissolve_segments = set()
    total_scene_dissolve_len = np.sum([dissolve_segment[1] - dissolve_segment[0] + 1 for dissolve_segment in true_scd])
    for scd in predicted_scd:
        for dissolve_segment in true_scd:
            if scd in range(dissolve_segment[0], dissolve_segment[1] + 1):
                tp += 1
                checked_dissolve_segments.add(tuple(dissolve_segment))
                break
        else:
            fp += 1
    fn = len(true_scd) - len(checked_dissolve_segments)
    tn = scene_len - total_scene_dissolve_len + len(true_scd) - tp - fp - fn
    return tp, fp, tn, fn

In [ ]:
def f1_score_dissolve(true_scd, predicted_scd, scene_len):
    tp, fp, tn, fn = calculate_matrix_dissolve(true_scd, predicted_scd, scene_len)
    precision_score = calculate_precision(tp, fp, tn, fn)
    recall_score = calculate_recall(tp, fp, tn, fn)
    if precision_score + recall_score == 0:
        return 0
    else:
        return 2 * precision_score * recall_score / (precision_score + recall_score)

In [ ]:
def run_scene_change_detector_all_video_dissolve(scene_change_detector, dataset_path):
    video_dataset = load_json_from_file(os.path.join(dataset_path, 'info.json'))
    param_log = {
        '_mean_f1_score': []
    }
    for video_info in tqdm(video_dataset, leave=False):
        frames = read_video(os.path.join(dataset_path, video_info['source']))
        video_len = video_info['len']
        true_scene_changes = load_json_from_file(os.path.join(dataset_path, video_info['scene_change']))
        
        predicted_scene_changes, _, _ = scene_change_detector(frames)
        param_log['f1_score_{}'.format(video_info['source'])] = f1_score_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        video_tp, video_fp, video_tn, video_fn = calculate_matrix_dissolve(
            true_scene_changes.get('dissolve', []),
            predicted_scene_changes,
            video_len
        )
        param_log['tp_{}'.format(video_info['source'])] = video_tp
        param_log['fp_{}'.format(video_info['source'])] = video_fp
        param_log['tn_{}'.format(video_info['source'])] = video_tn
        param_log['fn_{}'.format(video_info['source'])] = video_fn
        param_log['_mean_f1_score'].append(param_log['f1_score_{}'.format(video_info['source'])])
    param_log['_mean_f1_score'] = np.mean(param_log['_mean_f1_score'])
    return param_log

In [ ]:
video_dataset_path = 'train_dataset'

In [ ]:
#Протестируем разработанный вами метод
run_scene_change_detector_all_video_dissolve(scene_change_detector_dissolve, video_dataset_path)

## Немного об оценивании задания ##

Оценивание задания будет производиться по следующей схеме:  

Пусть на скрытой выборке по F-метрике вы получили X, лучшее решение получило Y.

1. Базовая часть оценивется как $$20 * \left(\frac{\max(0, X_{base} - 0.5)}{Y_{base} - 0.5}\right)^2 + Bonus_{base}$$ Бонусные баллы $Bonus$ можно получить за оригинальные идеи в задаче или в её реализации
2. Дополнительное задание оценивается как $$5 * \frac{\max(0, X_{add} - 0.1)}{Y_{add} - 0.1} + Bonus_{add}$$Процесс получения бонусных баллов аналогичен получению бонусных баллов в базовой части

## Ваши ощущения ##

*До дедлайна пару часов и вы никак не можете улучшить текущее решение? Или наоборот, вы всё сделали очень быстро? Опишите кратко ваши ощущения от задания - сколько времени вы потратили на задание, сколько вы потратили на изучение питона и установку необходимых библиотек, как быстро вы придумывали новые идеи и как они давали прирост по метрике и в целом насколько это задание вам понравилось и что хотели бы изменить/добавить.*